In [49]:
# Dependencies 
from docplex.mp.model import Model 
import numpy as np
import copy

### Exercise 5.6

In [117]:
def run_opt(limits, demands, prod_costs, purchase_costs, hold_costs): 
    
    # Create model 
    model = Model()

    # Create Decision vars
    a, c, h = [0], [0], [0]
    for i in range(1, 5): 
        a.append(model.continuous_var(name = f'a{i}', lb = 0, ub = limits[i]))
        c.append(model.continuous_var(name = f'c{i}', lb = 0))
        h.append(model.continuous_var(name = f'h{i}', lb = 0))

    # Demand constraints
    for i in range(1, 5): 
        model.add_constraint(a[i] + c[i] + h[i-1] >= demands[i])

    # Inventory constraints 
    for i in range(1, 5): 
        model.add_constraint(a[i] + c[i] + h[i-1] - demands[i] == h[i])


    # Define objective 
    objective = (np.array(prod_costs).T @ a) + (np.array(purchase_costs).T @ c) + (np.array(hold_costs).T @ h)
    model.minimize(objective)

    model.solve()
    model.print_solution()
    
    return model

#### Part (B)

In [118]:
# Run Part B
params = {
    'limits': [0, 160, 160, 160, 160],
    'demands': [0, 150, 160, 225, 180],
    'prod_costs': [0, 35, 35, 35, 35],
    'purchase_costs': [0, 50, 50, 50, 50],
    'hold_costs': [0, 5, 5, 5, 5]
}
model_b = run_opt(**params)

objective: 26250.000
  a1=160.000
  h1=10.000
  a2=160.000
  h2=10.000
  a3=160.000
  c3=55.000
  a4=160.000
  c4=20.000


We shoud produce, buy, and hold the following: 

- January: Produce 160 and buy 0 (hold 10)
- February: Produce 160 and buy 0 (hold 10)
- March: Produce 160 and buy 55
- April: Produce 160 and buy 20

The optimal cost is \$26,250

#### Part (C)

Let's just search over the possible strategies. We see that we should schedule maintenance in January because it has the lowest cost of \$26,295. There's really no need to do any sort of analysis of reduced costs because it's a small problem and not instructed :)

In [119]:
# Run Part C
params_c = copy.copy(params)

# Define new production limits for schedule
jan_limits = [0, 151, 160, 160, 160]
feb_limits = [0, 160, 153, 160, 160]
mar_limits = [0, 160, 160, 155, 160]

for limits, month in zip([jan_limits, feb_limits, mar_limits], ['Jan', 'Feb', 'Mar']): 
    
    params_c.update({'limits': limits})

    print(f'{month} maintenance:')
    run_opt(**params_c)

Jan maintenance:
objective: 26295.000
  a1=151.000
  h1=1.000
  a2=160.000
  h2=1.000
  a3=160.000
  c3=64.000
  a4=160.000
  c4=20.000
Feb maintenance:
objective: 26320.000
  a1=160.000
  h1=10.000
  a2=153.000
  h2=3.000
  a3=160.000
  c3=62.000
  a4=160.000
  c4=20.000
Mar maintenance:
objective: 26325.000
  a1=160.000
  h1=10.000
  a2=160.000
  h2=10.000
  a3=155.000
  c3=60.000
  a4=160.000
  c4=20.000


#### Part (D)

We could run a new optimization problem with a new set of decision variables, but that isn't necessary. We know that buying lamps from D will always be more expensive than producing it ourselves. In the first two months, we don't buy any from C and we produce up to the limit. In March, we purchase 55 lamps from company C, and in April, we purchase 20 lamps from company C. Buying from D is cheaper than buying from C so we want to buy as many as we can from D (50) instead of C in these two months. One approach is to buy 50 from units from D in March, and none in April.

We save 5 dollars per lamp for each of the 50 lamps = 250 dollar total savings!

#### Part (E)

This is effectively measuring the tradeoff between buying more units and holding them or just buying them as needed. We want to look at the reduced cost of buying in February to find the cuttoff value because this is the max it would have to be to make the decision var non-zero in the optimal simplex. Here, we see the reduced cost for buying from C in February is 5 dollars, so the discount must be at least this much to be attractive to us!

In [120]:
print(f'c2 reduced cost: {model.get_var_by_name("c2").reduced_cost}')

c2 reduced cost: 5.0


#### Part (F)

Beause we are checking how a change in a cost will affect optimality, we need to check the upper and lower bounds for maintaining optimality to see if we need to rerun a new problem. We see that the increase of 3 dollars does not change the basis, so the optimal cost will increase by 3 times the number of lamps we hold in February = 3 * 10 = 30. We verify this below by running a new model.

In [121]:
# Run Part F
params_f = copy.copy(params)
params_f.update({'hold_costs': [0, 5, 8, 5, 5]})
run_opt(**params_f)

objective: 26280.000
  a1=160.000
  h1=10.000
  a2=160.000
  h2=10.000
  a3=160.000
  c3=55.000
  a4=160.000
  c4=20.000


docplex.mp.Model['docplex_model60']

#### Part (G)

We can see that with this new demand, our current solution isn't optimal anymore, so we should just rerun a new model. See below that the new solution yields an optimal objective at 23,875

In [122]:
# Run Part G 
params_g = copy.copy(params)
params_g.update({'demands': [0, 90, 160, 225, 180]})
run_opt(**params_g)

objective: 23875.000
  a1=160.000
  h1=70.000
  a2=160.000
  h2=70.000
  a3=160.000
  h3=5.000
  a4=160.000
  c4=15.000


docplex.mp.Model['docplex_model61']

## Exercise 5.8

In [160]:
def run_opt58(clay, enamel, dry_room, kiln, p12, rhs, profits, dual_vars = True): 
    
    # Initialize Model
    model = Model()

    # Create Decision vars
    B = model.continuous_var(name = 'B', lb = 0)
    C = model.continuous_var(name = 'C', lb = 0)
    E = model.continuous_var(name = 'E', lb = 0)
    P1 = model.continuous_var(name = 'P1', lb = 0)
    P2 = model.continuous_var(name = 'P2', lb = 0)
    
    v = [E, C, P1, P2, B]
    
    # Define Constraints 
    clay_cts = model.add_constraint(np.array(clay).T @ v <= rhs[0])
    enamel_cts = model.add_constraint(np.array(enamel).T @ v <= rhs[1])
    dry_room_cts = model.add_constraint(np.array(dry_room).T @ v <= rhs[2])
    kiln_cts = model.add_constraint(np.array(kiln).T @ v <= rhs[3])
    p12_cts = model.add_constraint(np.array(p12).T @ [P1, P2] == rhs[4])
    
    # Define objective 
    model.maximize(np.array(profits).T @ v)
    
    model.solve()
    model.print_solution()
    
    if dual_vars: 
        cts = [clay_cts, enamel_cts, dry_room_cts, kiln_cts, p12_cts]
        print('Dual Values: ', model.dual_values(cts))
        
    return model

#### Part (A) 

The optimal solution is verified by Table 5.4, which is B = 5, C = 2 and all else = 0. The means we shoudl make 0 English sets, 0 Primrose sets, 2 Currier sets, and 5 Bluetail sets.The maximum profit we obtain from this is \$649.00

In [162]:
params = {
    'clay': [10, 15, 10, 10, 20], 
    'enamel': [1,2,2,1,1], 
    'dry_room': [3,1,6,6,3],
    'kiln': [2,4,2,5,3], 
    'p12': [1,-1], 
    'rhs': [130, 13, 45, 23, 0], 
    'profits': [51, 102, 66, 66, 89]
}
model_a = run_opt58(**params, dual_vars = False)

objective: 649.000
  B=5.000
  C=2.000


#### Part (B)

- Clay: For each pound of additional clay we have above our currrent limit of 130, we can increase our profit by \$1.429.
-  Enamel: For each pound of additional enamel we have above our currrent limit of 9, we can increase our profit by \$0. That is to say it would have no impact on our optimal profit.
-  Dry Room: For each hour of additional dry room use we could have above our currrent limit of 45, we can increase our profit by \$0.That is to say it would have no impact on our optimal profit.
- Kiln: For each hour of additional kiln use above our current limit of 23, we can increase our profit by \$20.143. This has a huge impact because it means we can process more!
- Primrose: For each unit of difference we allow between producing the types of Primrose (specicially make more using the first process), we can increase our profit by \$11.429. 

We see these dual values match that of the tables!

In [171]:
for i in range(5): 
    print(model.get_constraint_by_index(i).dual_value)

1.4285714285714286
0
0
20.142857142857142
11.42857142857143


#### Part (C)

Yes, for each pound we buy, we pay 1.1 but we expect to make 1.429. Now, 20 is within the allowable increase to remain optimal so these shadow prices hold. We should profit an extra $20(1.429-1.1) = \$6.58$

#### Part (D)

Note that the allowable decrease is 28 hours of dry room to remain optimal, so we can't directly find the decrease in profit using shadow prices as we did in (c). The shadow price is $0 which means it would not increase the profit under the current basis anyway, so we will rerun a new model. I didn't bother vectorizing this code because they already gave us the formulation.

In [178]:
params_d = copy.copy(params)
params_d.update({'rhs': [130, 13, 45 - 30, 23, 0]})
model_d = run_opt58(**params_d, dual_vars = False)

objective: 637.889
  B=4.111
  C=2.667


Our profit decreases by 11.11 (make a wish). This comes from the fact that our dual value for the dry room constraint is $5.55, which is the amount of profit we lose by decreasing our allowed use by 1 hour. We decrease by 2 above the limit (30-28=2) so we lose the expected 5.5555*2 = 11.11 roughly. So our upper bound is just at 0 if we are within 28 allowed decrease, and 5.555 dollar decrease for each new decrease beyond 28 hours. 

#### Part (E)

My current function doesn't allow for this, so I'll create a new one. We see below that at optimailty, the amount of P1 = 3.5, which is clearly positive....so yes.

In [188]:
def run_opt_part_e(clay, enamel, dry_room, kiln, p12, rhs, profits, dual_vars = True): 
    
    # Initialize Model
    model = Model()

    # Create Decision vars
    B = model.continuous_var(name = 'B', lb = 0)
    C = model.continuous_var(name = 'C', lb = 0)
    E = model.continuous_var(name = 'E', lb = 0)
    P1 = model.continuous_var(name = 'P1', lb = 0)
    P2 = model.continuous_var(name = 'P2', lb = 0)
    
    v = [E, C, P1, P2, B]
    
    # Define Constraints 
    clay_cts = model.add_constraint(np.array(clay).T @ v <= rhs[0])
    enamel_cts = model.add_constraint(np.array(enamel).T @ v <= rhs[1])
    dry_room_cts = model.add_constraint(np.array(dry_room).T @ v <= rhs[2])
    kiln_cts = model.add_constraint(np.array(kiln).T @ v <= rhs[3])
    p12_cts = model.add_constraint(np.array(p12).T @ [P1, P2] >= rhs[4])
    
    # Define objective 
    model.maximize(np.array(profits).T @ v)
    
    model.solve()
    model.print_solution()
    
    if dual_vars: 
        cts = [clay_cts, enamel_cts, dry_room_cts, kiln_cts, p12_cts]
        print('Dual Values: ', model.dual_values(cts))
        
    return model

In [189]:
model_e = run_opt_part_e(**params, dual_vars = False)

objective: 689.000
  B=4.000
  C=1.000
  P1=3.500
